In [3]:
import zoom_api
from pathlib import Path
import json

## Get Authentication Token

Note: Don't print this

In [4]:
credentials = zoom_api.Credentials.from_environment_variables()
oauth = credentials.get_oauth()
token = oauth["access_token"]
assert token

## Get Meetings

In [5]:
meetings = [json.loads(path.read_text()) for path in Path('data/meetings').glob('*.json')]
len(meetings)

### Filter out unique meetings

Some "meetings" have multiple sessions, but they share the same id number (for example, if someone registers for a 5-session python course).  No need to look up for the registrations for every single session, so let's just keep the set of ids already used.

In [6]:
print(f"""
Unique IDs: {len(set(map(lambda d: d['id'], meetings)))}
Total Meetings: {len(meetings)}
""")

In [8]:
from zoom_api import meeting_type_is, MeetingType
class_types = {MeetingType.RecurringFixedTime, MeetingType.RecurringNoFixedTime, MeetingType.Scheduled}
unique_meetings = list({m['id']: m for m in meetings if meeting_type_is(m, *class_types)}.values())
len(unique_meetings)

## Get Registrants for Each Meeting

In [ ]:
meeting = unique_meetings[1]
meeting.keys()

In [15]:
all_meeting_regs = {}
for meeting in unique_meetings:
    try:
        regs = zoom_api.list_registrants(token=token, meeting_id=meeting['id'])
        all_meeting_regs[meeting['id']] = regs
    except ValueError as exc:
        if 'Registration has not been enabled' in str(exc):
            pass
        else:
            raise exc
len(all_meeting_regs)

## Save Registration Data

In [26]:

folder = Path('data/registrations')
folder.mkdir(exist_ok=True, parents=True)
for meeting_id, registrations in all_meeting_regs.items():
    (folder / (f'registrations_for_meeting_{meeting_id}.json')).write_text(json.dumps(registrations, indent=4))
    
    

## Confirm Done

In [29]:
done_flag_filename = snakemake.output[0]
done_flag_filename

In [30]:
Path(done_flag_filename).touch(exist_ok=True)